# The IPython Lambda Notebook

Welcome to the [IPython Lambda Notebook](http://lambdanotebook.com/)!  This is a tool based on the [IPython Notebook](http://ipython.org/notebook.html), but tailored for doing calculations of the sort involved in linguistic semantics, especially involving the typed lambda calculus.  The aim is to provide facilities for writing 'digital fragments' -- working implementations of a compositional system developed in the course of semantic research.  The Lambda Notebook is very much in an early state, what might be called a public alpha, but because it is based on IPython, it inherits many extremely powerful tools aimed at scientific computing. 

 1. [Basics](#basics)
 2. [Background of IPython notebook](#interface)
 3. [Miscellany](#misc)
 
Section 2 is recommended reading if you have not used IPython before (and especially if you haven't used Python).  If you want to use the lambda notebook on a computer not connected to the internet, look at the first bullet in section 3.

You can proceed through this document by pressing shift+enter.

<a id='basics'></a>

## 1. Basics

IPython notebook is structured into cells.  Each cell contains code or text (typically markdown formatted text).  Code cells without a "magic" command on the first line, which would be indicated by `%%`, contain ordinary python code.  The value of the last line of any python cell, as long as it is not `None`, is displayed when the cell is executed.

In [ ]:
print("This is the output of a python print command.")
20 + 22

If the first line of the cell begins with `%%`, then the cell is interpreted differently, depending on the command following those symbols.  This is called a _cell magic_, and the Lambda Notebook makes heavy use of cell magics to interpret e.g. typed lambda calculus expressions.  
Despite the alpha status, there are already many things you can do related to compositional semantics.  The main cell magic is the `%%lamb` magic.  If a cell starts with `%%lamb`, then the rest of the cell will be interpreted as a series of definitions in the Lambda Notebook Metalanguage (LNM).

For example, here is how you can define some basic lexical entries in LNM, and use them:

In [ ]:
%%lamb
||every|| = lambda f_<e,t> : lambda g_<e,t> : Forall x_e : f(x) >> g(x)
||cat|| = lambda x_e : Cat_<e,t>(x_e)
||dance|| = lambda x_e : Dance_<e,t>(x_e)

Lexical entries defined this way are available in python.  The `*` operator performs semantic composition, so the three above lexical entries can be combined as follows:

In [ ]:
(every * cat) * dance

**Running code:** To (re-)run a cell you can hit the run button in the toolbar with the cell selected, or hit shift+enter while the cell is selected.  Both of the above two cells are runnable.

The `*` operator here is overloaded to do composition on lexical items (and their results).  We can see this by probing the result of `*` in various ways.  (The following cell is python code.)

In [ ]:
result = (every * cat) * dance
result.tree()

The lambda notebook also supports several "line magics", which can be mixed in with python code inside a cell.  This is indicated by a single `%` command at the beginning of a line.  The `%lamb` magic can be used as a line magic.  In addition, a single expression can be displayed using the `%te` line magic (standing for `TypedExpression`).  Note that these magics will only display output if they are the last thing in a cell.  The result of `%te` can be assigned to a python variable.

In [ ]:
%lamb ||dog|| = L x_e : Dog(x)

In [ ]:
%te (L f_<e,t> : L x_e : f(x))(L y_e : Dog(y))

In [ ]:
%te reduce (L f_<e,t> : L x_e : f(x))(L y_e : Dog(y))

In [ ]:
x = %te reduce (L f_<e,t> : L x_e : f(x))(L y_e : Dog(y))
x.type

The expression parser can be accessed in python mode via the function `te` by providing a string argument.  The resulting python objects can be examined in many interesting ways; one example is below.

In [ ]:
x = te("(L f_<e,t> : L x_e : f(x))(L y_e : Dog(y))")
x.type

In [ ]:
x.reduce_all().derivation.trace()

Please see the notebook [Lambda Notebook Demo.ipynb](http:///notebooks/Lambda%20Notebook%20Demo.ipynb) for many more examples of how to use these two modes.  Two reasonably detailed examples of full, research-oriented fragments can be found in [fragments/Continuations and quantifier scope.ipynb](http:///notebooks/fragments/Continuations%20and%20quantifier%20scope.ipynb) and in [fragments/Variable free binding.ipynb](http:///notebooks/fragments/Variable%20free%20binding.ipynb).  A good starting point for understanding and extending the lambda notebook is [tutorials/Type shifting.ipynb](http:///notebooks/tutorials/Type%20shifting.ipynb), which goes through the process of adding a variety of type-shifting operations to a compositinoal system.  Many further examples of various things you might want to do are provided as well, and we welcome contributions.

These documentation cells are in another mode where you can write ['markdown'-formatted](http://daringfireball.net/projects/markdown/) text; this mode can be chosen from the dropdown box in the toolbar.  Also, the IPython notebook uses the MathJax library to support latex math mode; most equations can be written simply be wrapping them in dollar signs.  For example: '$\lambda x : x \in D_e \:.\: x\text{ is a cat}$'.  To see what the markdown looks like in this notebook, just double-click on any of the text cells (or hit enter with the cell selected).  To re-render it, hit shift-enter.

### The state of the lambda notebook

Some things that are implemented:

 * lambda calculus, types, etc
 * composition operations on a layer above the pure metalanguage, including the ability to add new ones
 * detailed display of compositional derivations
 * multiple composition paths
 * type shifting, ability to add new type shifts
 * composition operations on tree structures (partial).  (The normal mode illustrated above operates bottom-up on lexical items, building tree structures in tandem, i.e. a species of strictly-mapped categorial grammar.)
 * binding (partial)
 
Some things that are prominently missing, ordered from least to most difficult:

 * Some kind of implementation of projective meanings (for presuppositions)
 * Transformations on syntactic representations, for e.g. QR.  (You can do QR now, you just have to manually construct the LFs.)
 * Theorem proving on the logical subset of the metalanguage.  (Also, no model theory.)
 * Integration with any parsing of natural language.  (E.g., compare the web demo of boxer.)
 
Contributions are welcome!  The first two of these are preconditions for moving to beta and currently under development, the latter two are further away.

<a id="interface"></a>

## 2. Interface and background on IPython

The Lambda Notebook is based heavily on the IPython Notebook and most of the interface you are seeing is IPython.  Because the IPython Notebook is purely browser-based, its interface may be somewhat idiosyncratic compared to native apps that you are used to.  We are working on a closer native wrapping at least for OS X, but in the meantime here are some things that may be helpful in understanding what is going on for users who are new to IPython.

 * IPython is an enhanced interface to python that supports many extremely useful things; one of the main targets is interfacing with the 'scipy stack', for doing scientific and mathematical computing of the sort often done in R and matlab.  It also provides good support for distributed computing.
 * IPython Notebook is a particular interface to IPython inspired by mathematica notebooks, and is centered around interleaving code with documentation.
 * The IPython Notebook runs as a server, accessible only from your local computer (at least by default).  You then select and edit notebooks via a web browser that contacts this server.  Typically it runs at the URL [http://localhost:8888](http://localhost:8888), and you can access this from any browser as long as the server is running.
 * The server is represented by a bunch of logging messages in a window, either as part of an app (on the mac app version) or in a terminal/console, if running directly.  To shut it down, (i) if you are running the mac app, hit `cancel`, or (ii) if you are running from a terminal/console, hit Ctrl-C twice.
 * Every notebook corresponds to both a document and a 'kernel' which runs the code on the server.  If you close a document by closing the browser window, the kernel will still be running and can be shut down (or re-accessed) at the notebook browser.  Kernels store program state of any code executed in cells.
 * The default location for files is `~/Documents/lambda_notebook/`, where `~` represents your home directory.  If the .app is in a folder that contains a directory "notebooks", it will use that instead.
 * Though IPython notebook doesn't (currently) have a way of creating directories via the browser interface, you can make directories in the normal way (e.g. in Finder or at the command line) in this folder and access them with the notebook browser.
 * Because of all this complexity, there is currently no way to double click in your OS on a notebook and open it.  Rather, you will need to copy the notebook into this Documents directory, run the notebook, and open the file using the notebook browser.
 * Currently, lambda notebook .ipynb files will not work in vanilla ipython notebook unmodified.  However, if you want to do this adding a cell with the following code and running it will be enough: `import lamb; import lamb.lnsetup; lamb.lnsetup.ipython_setup()`.  (Well, you must have `lamb` in your python path somehow.)
 
We welcome patches to reduce any and all of this complexity!

<a id="misc"></a>

## 3. Miscellaneous useful things

 * If you accidentally mess up any of the default install files from the mac app, you can find them again inside the app package (or just download them from the repository).  If you delete your notebook directory entirely it will be recreated on running the app.
 * The lambda notebook infrastructure can be loaded anywhere.  Two things won't generally work: the cell magic (which should work in all IPython cells but isn't well-tested, and won't work outside of IPython) and the fancy equation display, which requires the browser.  To do this, as long as the `lamb` folder is in the python path somehow, just do `import lamb`.  In the module `lamb.lnsetup` are some helper functions for setup, including the functions `ipython_setup()` and `reload_lamb()` which may be useful to look at for configuration purposes.
 * to run the mac app at the command line, use the following command relative to the location of the app: `./Lambda\ Notebook.app/Contents/Resources/start_notebook.app/Contents/MacOS/start_notebook`.  Though if you're at this point I reccommend a full IPython install instead.
 * To see the source for the `lamb` module in the mac app, you can look inside the app package, in particular at: `Lambda\ Notebook.app/Contents/Resources/start_notebook.app/Contents/Resources/lib/python3.3/lamb/`.  Or, obviously, just download it from github.  Again, if you're at the point of caring about this I recommend a full IPython install.